In [ ]:
import numpy as np
import pandas as pd

import torch
import torchvision.transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image
from enum import Enum

DATA_FOLDER = "/kaggle/input/cs-480-2024-spring/data"

class DataSource(Enum):
    TRAIN = 1
    TEST = 2

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
    
def fetch_image_as_tensor(id: int, image_type: DataSource):
    image_folder = "train_images" if image_type == DataSource.TRAIN else "test_images"
    
    return transform(Image.open(f"{DATA_FOLDER}/{image_folder}/{id}.jpeg"))

def reject_outliers(data, m=3):
    results = np.log10(data[:, 164:])

    return data[np.all(abs(results - np.mean(results, axis=0)) < m * np.std(results, axis=0), axis=1)]

def model_output_to_traits(output, train_min, train_max):
    return 10 ** (output * (np.log10(train_max) - np.log10(train_min)) + np.log10(train_min))

class TrainDataset(Dataset):
    def __init__(self, data, train_min, train_max):
        self.data = data
        self.log_train_min = np.log10(train_min)
        self.log_train_max = np.log10(train_max)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, row_index):
        row = self.data[row_index]
        index = int(row[0])
        image = fetch_image_as_tensor(index, DataSource.TRAIN)
        aux_data = torch.tensor(row[1:164]).float()
        
        result = torch.tensor((np.log10(row[164:]) - self.log_train_min) / (self.log_train_max - self.log_train_min)).float()
        
        return image, aux_data, result
    
np.random.seed(0)
    
batch_size = 128

known_values = pd.read_csv(f"{DATA_FOLDER}/train.csv").values
filtered_known_values = reject_outliers(known_values)

indices = np.random.permutation(filtered_known_values.shape[0])
split_index = int(0.8 * len(filtered_known_values))
train_idx, validation_idx = indices[:split_index], indices[split_index:]
train_subset, validation_subset = filtered_known_values[train_idx,:], filtered_known_values[validation_idx,:]

TRAIN_MIN = np.min(train_subset[:, 164:], axis=0)
TRAIN_MAX = np.max(train_subset[:, 164:], axis=0)

train_data_loader = DataLoader(TrainDataset(train_subset, TRAIN_MIN, TRAIN_MAX), batch_size=batch_size)
validation_data_loader = DataLoader(TrainDataset(validation_subset, TRAIN_MIN, TRAIN_MAX), batch_size=batch_size)

In [ ]:
import torch.nn as nn
import torchvision.models as models

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.cnn = models.resnet152()
        
        self.aux_linear_relu_stack = nn.Sequential(
            nn.Linear(163, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
        )
        
        self.combined_linear_relu_stack = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.cnn.fc.out_features + 256, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Dropout(0.5),
             nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
             nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
             nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Linear(4096, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 6),
        )
        
    def forward(self, image, aux_data):
        x1 = self.cnn(image)
        x2 = self.aux_linear_relu_stack(aux_data)
        
        x = torch.cat((x1, x2), dim=1)
        x = self.combined_linear_relu_stack(x)
        return x
        
model = NeuralNetwork().to(device)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, aux_data, y) in enumerate(dataloader):
        x, aux_data, y = x.to(device), aux_data.to(device), y.to(device)

        # Compute prediction error
        pred = model(x, aux_data)
        loss = loss_fn(pred, y)
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def validate(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0

    with torch.no_grad():
        for x, aux_data, y in dataloader:
            num_batches += 1
            x, aux_data, y = x.to(device), aux_data.to(device), y.to(device)
            pred = model(x, aux_data)
            
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches

    print(f"Avg loss: {test_loss:>8f} \n")
    
    return test_loss
    
def test():
    test_values = pd.read_csv(f"{DATA_FOLDER}/test.csv").values

    with torch.no_grad():
        final_predictions = np.array([])
        for row in test_values:
            index = int(row[0])
            image = fetch_image_as_tensor(index, DataSource.TEST).unsqueeze(0)
            aux_data = torch.tensor(row[1:164]).float().unsqueeze(0)

            image, aux_data = image.to(device), aux_data.to(device)
            pred = model(image, aux_data)
            pred_np = model_output_to_traits(pred.cpu().numpy()[0], TRAIN_MIN, TRAIN_MAX)

            if final_predictions.any():
                final_predictions = np.append(
                    final_predictions, 
                    [np.concatenate(([index], pred_np))],
                    axis=0
                )
            else:
                final_predictions = np.array([np.concatenate(([index], pred_np))])

        df = pd.DataFrame(final_predictions)

        df.to_csv("predictions.csv",index=False)  

In [ ]:
#20

epochs = 20

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data_loader, model, loss_fn, optimizer)
    validate(validation_data_loader, model, loss_fn)

test()
            
print("Done!")

In [ ]:
with torch.no_grad():
    for x, aux_data, y in validation_data_loader:
        x, aux_data, y = x.to(device), aux_data.to(device), y.to(device)
        pred = model(x, aux_data)
        
        print(model_output_to_traits(pred.cpu(), TRAIN_MIN, TRAIN_MAX)[0:4])
        print(model_output_to_traits(y.cpu(), TRAIN_MIN, TRAIN_MAX)[0:4])
        break

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")